In [4]:
import pandas as pd

In [31]:
def process_data_file(fname):
    # Load dataset initially with all values as tring objects and skip first three rows under the header row
    dtypes = {'TBLID': 'string', 'GEOID': 'string', 'GEONAME': 'string', 'PROFLN': 'float64', 'TITLE': 'string', 'ESTIMATE_1': 'string', 'MG_ERROR_1': 'string',
    'ESTIMATE_2': 'string', 'MG_ERROR_2': 'string', 'ESTIMATE_3': 'string', 'MG_ERROR_3': 'string', 'ESTIMATE_4': 'string', 'MG_ERROR_4': 'string', 'ESTIMATE_5': 'string',
    'MG_ERROR_5': 'string', 'ESTIMATE_6': 'string', 'MG_ERROR_6': 'string', 'ESTIMATE_7': 'string', 'MG_ERROR_7': 'string', 'ESTIMATE_8': 'string', 'MG_ERROR_8': 'string',
    'OCC_CODE': 'string', 'OCC_DESCRIPTION': 'string'}
    df = pd.read_csv(fname, skiprows=lambda r: r in [1,2,3], dtype=dtypes)

    # Drop TBLID as it is of no use here
    df.drop(columns=['TBLID'], inplace=True)

    # Exclude rows where Title is not 'Number' or 'Percent' as there is no values in the EST and MOE fields for such rows
    df_new = df[ (df.TITLE == 'Number') | (df.TITLE == 'Percent') ]

    # Unpivot DataFrame from wide to long format leaving geography, occupation, profile number, and title as the identifier set
    df_new_melted = df_new.melt(id_vars=['GEOID', 'GEONAME', 'OCC_CODE', 'OCC_DESCRIPTION', 'PROFLN', 'TITLE'], var_name='ATTRIBUTE', value_name='VALUE')

    # Caputre report type in a column for use in attribute code
    df_new_melted['REPORT_TYPE'] = ['EST' if 'EST' in rt else 'MOE' for rt in df_new_melted.ATTRIBUTE]

    # Add an attribute code columm (for use in relating to 2014_2018_EEOALL1R_Column_Headers)

    attr_re_map = {'ESTIMATE_1': 7, 'MG_ERROR_1': 7, 'ESTIMATE_2': 1, 'MG_ERROR_2': 1, 'ESTIMATE_3': 6, 'MG_ERROR_3': 6, 'ESTIMATE_4': 4, 'MG_ERROR_4': 4, 'ESTIMATE_5': 2, 'MG_ERROR_5': 2, 'ESTIMATE_6': 3, 'MG_ERROR_6': 3, 'ESTIMATE_7': 5, 'MG_ERROR_7': 5, 'ESTIMATE_8': 0, 'MG_ERROR_8': 0}
    # Define a function to produce attribute codes from df
    def concat_attr_code(report_type, gender_num, val_format, attr):
        gender_num_map = {1: 'B', 2: 'B', 3: 'M', 4: 'M', 5: 'F', 6: 'F'}
        val_format_map = {'Number': 'N', 'Percent': 'P'}
        attr_tuple = (report_type, gender_num_map[gender_num], val_format_map[val_format], str(attr_re_map[attr]))
        attr_code = '_'.join(attr_tuple)
        return attr_code
    df_new_melted['ATTR_CODE'] = df_new_melted.apply(lambda row: concat_attr_code(row['REPORT_TYPE'], row['PROFLN'], row['TITLE'], row['ATTRIBUTE']), axis=1)

    # Drop columns used for attribute code as they are no longer needed
    df_new_melted.drop(columns=['PROFLN', 'TITLE', 'ATTRIBUTE', 'REPORT_TYPE'], inplace=True)

    return df_new_melted


In [32]:
nation_processed_df = process_data_file('2014_2018_EEOALL1R_010_Nation.csv')

In [34]:
state_processed_df = process_data_file('2014_2018_EEOALL1R_040_State.csv')

In [45]:
cbsa_processed_df = process_data_file('2014_2018_EEOALL1R_310_CBSA.csv')

In [46]:
countySet_processed_df = process_data_file('2014_2018_EEOALL1R_902_CountySets.csv')

In [49]:
nation_processed_df.to_csv('2014_2018_EEOALL1R_010_Nation_processed.csv', index=False)
state_processed_df.to_csv('2014_2018_EEOALL1R_040_State_processed.csv', index=False)
cbsa_processed_df.to_csv('2014_2018_EEOALL1R_310_CBSA_processed.csv', index=False)
countySet_processed_df.to_csv('2014_2018_EEOALL1R_902_CountySets_processed.csv', index=False)

In [48]:
def print_geo_record_count(df, df_name):
    print('{1} dataset has {0:,} rows.'.format(df.shape[0], df_name))
    return
print_geo_record_count(nation_processed_df, 'Nation')
print_geo_record_count(state_processed_df, 'State')
print_geo_record_count(cbsa_processed_df, 'Core Based Statistical Area')
print_geo_record_count(countySet_processed_df, 'County Set')

Nation dataset has 22,848 rows.
State dataset has 1,188,096 rows.
Core Based Statistical Area dataset has 12,337,920 rows.
County Set dataset has 33,700,800 rows.
